In [ ]:
# 2_DeepLabV3_Training.ipynb

import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, AveragePooling2D, UpSampling2D, concatenate
from tensorflow.keras.optimizers import Adam

# --- CONFIGURATION ---
IMG_HEIGHT = 256
IMG_WIDTH = 256
DATA_PATH = '../data/train'
MODEL_SAVE_PATH = '../saved_models/deeplabv3_oil_spill.h5'

# --- DATA LOADER (Included here to ensure standalone execution) ---
def load_data(path):
    images = []
    masks = []
    img_dir = os.path.join(path, 'images')
    mask_dir = os.path.join(path, 'labels') 

    if not os.path.exists(mask_dir):
        mask_dir = os.path.join(path, 'masks') # Check alternate name

    files = os.listdir(img_dir)[:500]
    for file_name in files:
        img_path = os.path.join(img_dir, file_name)
        img = cv2.imread(img_path)
        if img is None: continue

        # Try exact name or PNG variant
        mask_path = os.path.join(mask_dir, file_name)
        if not os.path.exists(mask_path):
            mask_path = os.path.join(mask_dir, os.path.splitext(file_name)[0] + ".png")
            
        mask = cv2.imread(mask_path, 0)
        if mask is None: continue

        img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
        mask = cv2.resize(mask, (IMG_WIDTH, IMG_HEIGHT))
        images.append(img)
        masks.append(mask)

    images = np.array(images) / 255.0
    masks = np.array(masks) / 255.0
    masks = np.expand_dims(masks, axis=-1)
    return images, masks

# Load and Split
try:
    X, y = load_data(DATA_PATH)
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"Loaded {len(X)} images.")
except Exception as e:
    print(f"Data loading failed: {e}")

# --- DEEPLAB V3+ BUILDING BLOCKS ---
def convolution_block(block_input, num_filters=256, kernel_size=3, dilation_rate=1, padding="same", use_bias=False):
    x = Conv2D(num_filters, kernel_size=kernel_size, dilation_rate=dilation_rate, padding="same", use_bias=use_bias)(block_input)
    x = BatchNormalization()(x)
    return Activation("relu")(x)

def DilatedSpatialPyramidPooling(dspp_input):
    dims = dspp_input.shape
    x = convolution_block(dspp_input, kernel_size=1, dilation_rate=1)
    out_6 = convolution_block(dspp_input, kernel_size=3, dilation_rate=6)
    out_12 = convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
    out_18 = convolution_block(dspp_input, kernel_size=3, dilation_rate=18)
    
    x_pool = AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
    x_pool = convolution_block(x_pool, kernel_size=1, use_bias=True)
    x_pool = UpSampling2D(size=(dims[-3] // x_pool.shape[1], dims[-2] // x_pool.shape[2]), interpolation="bilinear")(x_pool)

    x = concatenate([x, out_6, out_12, out_18, x_pool])
    return x

def DeepLabV3Plus(input_shape):
    inputs = Input(input_shape)
    # Encoder
    x = Conv2D(64, (3, 3), padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(64, (3, 3), strides=2, padding="same")(x)
    
    x = Conv2D(128, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Conv2D(128, (3, 3), strides=2, padding="same")(x)
    
    # Bridge & Decoder
    x = DilatedSpatialPyramidPooling(x)
    x = UpSampling2D((4, 4), interpolation="bilinear")(x)
    
    # Output
    x = Conv2D(1, (1, 1), padding="same")(x)
    x = Activation("sigmoid")(x)

    model = Model(inputs, x)
    return model

# Build, Compile, Train
if 'X_train' in locals():
    model = DeepLabV3Plus((IMG_HEIGHT, IMG_WIDTH, 3))
    model.compile(optimizer=Adam(learning_rate=0.001), loss="binary_crossentropy", metrics=["accuracy"])
    
    print("Starting DeepLab Training...")
    history = model.fit(X_train, y_train, batch_size=8, epochs=20, validation_data=(X_val, y_val))
    model.save(MODEL_SAVE_PATH)
    print(f"DeepLab model saved to {MODEL_SAVE_PATH}")
else:
    print("Data not loaded, skipping training.")